In [167]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [168]:
pd.set_option('display.max_columns', None)
flats = pd.read_csv('~/DS_Bootcamp/FlatPrices/data/_data.csv',index_col=0)

In [169]:
#Первый признак - комнаты, разобьем их на категории: числовое количество комнат (0 - не определено) и one-hot кодированием признаков смежности или изолированности 

#Заполним число комнат - nan нулями, разбив во вторую колонку данные по комнатам 
rooms = pd.DataFrame(flats['Количество комнат'].fillna('0').map(lambda x: map(str.strip,x.split(','))).to_list(),columns=['Num_Rooms','Room_Type'])

#get_dummies разобьет на три колонки из которых приведем к двум логической алгеброй
RoomDummie = pd.get_dummies(rooms['Room_Type'])
rooms['Num_Rooms'] = rooms['Num_Rooms'].map(int)
RoomDummie['Изолированная'] = RoomDummie['Изолированная'] + RoomDummie['Оба варианта']
RoomDummie['Смежная'] = RoomDummie['Смежная'] + RoomDummie['Оба варианта']

#applymap оказывается будет deprecated
RoomDummie = RoomDummie.drop(columns='Оба варианта').map(int).rename(columns={'Изолированная':'has_isolated','Смежная':'has_adjacent'})
RoomsFinal = pd.concat([rooms,RoomDummie],axis=1).drop(columns='Room_Type')

In [170]:
flats = pd.concat([RoomsFinal,flats.loc[:,'Тип':]],axis=1)
#Убрали ID, уберем остальные колонки не имеющие числового смысла
dropcolumns = ['Тип','Телефоны','Описание'] #тип только квартиры -нет ни нан ни других типов
flats = flats.drop(columns=dropcolumns,axis=1)

colforrename = flats.columns[3:10]
flats = flats.rename(columns=dict(zip(colforrename,['Time_to_subway','Adress','Square_m2','HouseType','Parking','Price','Renovation'])))


In [171]:
#Обработка времени до метро - nan и none заменены -1, остальные числами
flats['Time_to_subway'] = flats['Time_to_subway'].fillna('-1').map(lambda x: re.search(r'(\d+) мин',x).group(1) if re.search(r'(\d+) мин',x) else '-1').astype(int)


In [172]:
#Обработка площади - берем общую
flats['Square_m2'] = flats['Square_m2'].str.split('/').map(lambda x: x[0]).astype('float')
flats['Renovation'] = flats['Renovation'].fillna('не указано')


## Пока таблица выглядит следующим образом

In [173]:
flats['Parking'] = flats['Parking'].fillna('не указано')
flats.head(10)

,Num_Rooms,has_isolated,has_adjacent,Time_to_subway,Adress,Square_m2,HouseType,Parking,Price,Renovation,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,4,0,0,9,"Москва, улица Новый Арбат, 27",200.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,4,0,0,8,"Москва, улица Новый Арбат, 27",198.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,4,1,1,7,"Москва, улица Новый Арбат, 27",200.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,4,1,1,3,"Москва, переулок Плотников, 21С1",170.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,2,0,0,7,"Москва, улица Новый Арбат, 15",58.0,"12/26, Панельный",не указано,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615
5,3,0,0,5,"Москва, 1-й Смоленский переулок, 21",92.0,3/7,не указано,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Smolensky De Luxe,NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/274837728
6,5,1,1,4,"Москва, переулок Романов, 3С1",213.0,4/5,наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...",Евроремонт,43-25-25-25-22,NaN,На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/273643908
7,3,0,0,3,"Москва, улица Арбат, 43С3",98.0,"2/4, Монолитный",подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Евроремонт,26 22 15,NaN,Во двор,"Совмещенный (1), Раздельный (1)","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),Нет,https://www.cian.ru/rent/flat/274475342
8,3,0,0,9,"Москва, Новинский бульвар, 18С1",120.0,"5/10, Сталинский",открытая,"130000.0 руб./ За месяц, Залог - 130000 руб., ...",Евроремонт,45 25 25,NaN,На улицу,Совмещенный (1),Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",NaN,NaN,3.0,Пасс (1),Нет,https://www.cian.ru/rent/flat/273973191
9,4,0,0,10,"Москва, улица Арбат, 30/3С1",90.0,"2/7, Сталинский",открытая,"210000.0 руб./ За месяц, Залог - 210000 руб., ...",Евроремонт,16 14 16 20,Балкон (1),На улицу и двор,"Совмещенный (1), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,310.0,Пасс (1),Да,https://www.cian.ru/rent/flat/272900409


In [174]:
#flats['Площадь, м2'].str.split('/',expand=True)
HouseParams = flats['Дом'].str.split('[/,]',regex=True,expand=True).rename(columns=dict(zip(range(3),['Flat_floor','Max_floors','HouseType'])))
HouseParams.head(10)

KeyError: 'Дом'

In [ ]:

#Наверное можно было и проще...) Вообще, если написать полное выражение для всей строки, то методом groupdict можно получить
#готовый словарь, но поскольку строки очень разные написать полное выражение без ошибок у меня не получилось 
Rent = lambda x: re.search(r'(?P<rent>\d+)\.\d руб\./',x)
Period = lambda x: re.search(r'/ За (?P<period>\w+),',x)
CommSumm = lambda x: re.search(r'Сумма коммунальных платежей - (?P<commsumm>\d+\.\d) руб\.',x)
Deposit = lambda x: re.search(r'Залог - (?P<deposit>\d+) руб\.,',x)
CommIncluded = lambda x: re.search(r'Коммунальные услуги (?P<comminc>\w+),',x)
RentTime = lambda x: re.search(r'Срок аренды - (.+),|Срок аренды - (.+)$',x)
Prepayment = lambda x: re.search(r'Предоплата (?P<prepayment>\d+) мес',x)

Price = flats['Цена']
funcdict = {'rent':Rent,'period':Period,'commsumm':CommSumm,'deposit':Deposit,'comminc':CommIncluded,'renttime':RentTime,'prepayment':Prepayment}
reslist = []
for elem in Price:
    res_dict = {}
    for key,value in funcdict.items():
        res_dict[key] = value(elem).group(1) if value(elem) else None 
    reslist.append(res_dict)

Price = pd.concat([Price,pd.DataFrame(reslist)],axis=1)
#Price[(Price['renttime'].isna()) & (Price['Цена'].map(lambda x: bool(re.search('Срок аренды',x))))]

# i = 0
# for elem in flats['Цена']:
#     try:
#         print(i, RentPeriod(elem), CommSumm(elem))
#     except:
#         print(i, elem)
#     i += 1